# Ответы на вопросы:

1. Сколько в списке пар слов синонимов: **8**

2. Сколько в списке пар слов, связанных отношением гипоним-гипероним: **16**

3. Сколько в списке пар слов связано другими типами отношений: **3**

4. Средний вес близости у всех трех групп:

    Синонимы: **8.772**

    Гипонимы и гиперонимы: **8.156**

    Другие типы отношений: **7.563**
    
Также посчитано среднее сходство слов, для которых не нашлось никакого отношения: **4.673**

Среднее сходство пар внутри каждой из групп и полные списки пар, связанные заданным типом отношения -- в конце ноутбука

In [1]:
!pip -q install nltk

In [2]:
!wget -q http://alfonseca.org/pubs/ws353simrel.tar.gz

In [22]:
!tar  xf ws353simrel.tar.gz

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/c204/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
from nltk.corpus import wordnet as wn
from typing import Dict, List, Tuple
import itertools

In [6]:
def load_similarity_pairs(inp_path: str, sep='\t'):
    word_similarity_triples: List[Tuple[str, str, float]] = []
    with open(inp_path, 'r', encoding="utf-8") as inp_file:
        for line in inp_file:
            attrs = line.strip().split(sep)
            word_1 = attrs[0] # .lower()
            word_2 = attrs[1] # .lower()
            similarity = float(attrs[2])
            word_similarity_triples.append((word_1, word_2, similarity))
    return word_similarity_triples

In [7]:
def synsets_list2lemma_list(synsets_list, ):
    lemmas = set()
    for synset in synsets_list:
        # Получение списка лемм конкретного синсета
        synset_lemmas = set((x.lower() for x in synset.lemma_names()))
        # Обновление списка лемм леммами конкретного синсета
        lemmas.update(synset_lemmas)
    return lemmas
        

In [8]:
def process_synsets_by_name(wn, name: str):
    synset_relation_lemmas = {}

    # Получаем список возможных синонимов слова, используя функцию synonyms()
    synset_relation_lemmas["synonym"] = [synonym for synonym in itertools.chain(*wn.synonyms(name))]
    # Получение списка синсентов, среди имён которых содержится данной слово
    possible_synsets_of_name = wn.synsets(name)
    
    hypernym_lemmas = set()
    hyponym_lemmas = set()
    meronym_lemmas = set()
    holonym_lemmas = set()
    antonym_lemmas = set()
    drf_lemmas = set()
    pertainym_lemmas = set()
    # Обход списка синсетов
    for synset in possible_synsets_of_name:
        if name not in synset.lemma_names():
            continue
        synset_name = synset.name()
        # Обработка гипонимов
        hypo_synsets = synset.hyponyms()
        hypo_synset_lemmas = synsets_list2lemma_list(hypo_synsets, )
        hyponym_lemmas.update(hypo_synset_lemmas)
            
        # Обработка гиперонимов
        hyper_synsets = synset.hypernyms()
        hyper_synset_lemmas = synsets_list2lemma_list(hyper_synsets, )
        hypernym_lemmas.update(hyper_synset_lemmas)
        
        # Обработка part-меронимов 
        part_meronym_synsets =  synset.part_meronyms()
        part_meronym_synset_lemmas = synsets_list2lemma_list(part_meronym_synsets, )
        meronym_lemmas.update(part_meronym_synset_lemmas)
        
        # Обработка member-меронимов 
        member_meronym_synsets =  synset.member_meronyms()
        member_meronym_synset_lemmas = synsets_list2lemma_list(member_meronym_synsets, )
        meronym_lemmas.update(member_meronym_synset_lemmas)
        
        # Обработка substance-меронимов 
        substance_meronym_synsets =  synset.substance_meronyms()
        substance_meronym_synset_lemmas = synsets_list2lemma_list(substance_meronym_synsets, )
        meronym_lemmas.update(substance_meronym_synset_lemmas)
        
        # Обработка part-холонимов 
        part_holonym_synsets =  synset.part_holonyms()
        part_holonym_synset_lemmas = synsets_list2lemma_list(part_holonym_synsets, )
        holonym_lemmas.update(part_holonym_synset_lemmas)
        
        # Обработка member-холонимов 
        member_holonym_synsets =  synset.member_holonyms()
        member_holonym_synset_lemmas = synsets_list2lemma_list(member_holonym_synsets, )
        holonym_lemmas.update(member_holonym_synset_lemmas)
        
        # Обработка substance-холонимов 
        substance_holonym_synsets =  synset.substance_holonyms()
        substance_holonym_synset_lemmas = synsets_list2lemma_list(substance_holonym_synsets, )
        holonym_lemmas.update(substance_holonym_synset_lemmas)
    

        # Обработка отношений, заданных только для синонимов: antonyms, derivationally_related_forms and pertainyms
        synset_lemma_id = f"{synset_name}.{name}"
        synset_lemma = wn.lemma(synset_lemma_id)
        # Обработка лемм-антонимов
        lemma_level_antonym_lemma_ids = synset_lemma.antonyms()
        lemma_level_antonym_lemma_names = [lemma_id.name().split('.')[-1].lower() for lemma_id in lemma_level_antonym_lemma_ids]
        antonym_lemmas.update(lemma_level_antonym_lemma_names)

        # Обработка лемм с отношением "pertainym"
        lemma_level_pertainym_lemma_ids = synset_lemma.pertainyms()
        lemma_level_pertainym_lemma_names = [lemma_id.name().split('.')[-1].lower() for lemma_id in lemma_level_pertainym_lemma_ids]
        pertainym_lemmas.update(lemma_level_pertainym_lemma_names)

        # Обработка лемм с отношением "derivationally_related_forms"
        lemma_level_drf_lemma_ids = synset_lemma.derivationally_related_forms()
        lemma_level_drf_lemma_names = [lemma_id.name().split('.')[-1].lower() for lemma_id in lemma_level_drf_lemma_ids]
        drf_lemmas.update(lemma_level_drf_lemma_names)
    
    other_lemmas = set()
    other_lemmas.update(meronym_lemmas)
    other_lemmas.update(holonym_lemmas)
    other_lemmas.update(antonym_lemmas)
    other_lemmas.update(drf_lemmas)
    other_lemmas.update(pertainym_lemmas)
    synset_relation_lemmas["hyponym-hypernym"] = hypernym_lemmas.union(hyponym_lemmas)
    # synset_relation_lemmas["hyponym"] = hyponym_lemmas
    # synset_relation_lemmas["meronym-holonym"] = meronym_lemmas.union(holonym_lemmas)
    # synset_relation_lemmas["holonym"] = holonym_lemmas
    synset_relation_lemmas["other"] = antonym_lemmas.union(drf_lemmas).union(pertainym_lemmas)
    # synset_relation_lemmas["antonym"] = antonym_lemmas
    # synset_relation_lemmas["derivationally_related_forms"] = drf_lemmas
    # synset_relation_lemmas["pertainym"] = pertainym_lemmas
    
    return synset_relation_lemmas

In [9]:
def find_relation_synsets_for_words(wn, words_list):
    # Для каждого слова из заданного списка функция находит слова,
    # связанные с заданным некоторым отношением
    word_relation_info = {}
    for word in words_list:
        synset_relation_lemmas_dict = process_synsets_by_name(wn, word)
        word_relation_info[word] = synset_relation_lemmas_dict
    return word_relation_info

In [10]:
def create_vocab_from_similarity_triples(triples):
    # Получение списка уникальных слов для слов из списка wordsim353
    vocab = set()
    for (word_1, word_2, sim) in triples:
        vocab.add(word_1)
        vocab.add(word_2)
    return vocab

In [11]:
def group_sim_pairs_by_rel_type(sim_triples, word_relation_info):
    # Для каждой пары слов из wordsim353 функция находит тип отношения между ними.
    # Пары слов группируются по типу отношения. Отсутствие отношения также запоминается
    rel_types = ["synonym", "hyponym-hypernym", "other"]
    word_pairs_grouped_by_rel_type = {}
    word_pairs_grouped_by_rel_type = {r_t: set() for r_t in rel_types}
    rel_types = tuple(word_pairs_grouped_by_rel_type.keys())
    for t in sim_triples:
        (word_1, word_2, similarity) = t
        (word_1, word_2) = (word_1, word_2) if word_1 < word_2 else (word_2, word_1)
        assert '||' not in  word_1 and '||' not in  word_2
        found_rel = False
        for rel_type in rel_types:
            if word_2.lower() in word_relation_info[word_1][rel_type] or \
                    word_1.lower() in word_relation_info[word_2][rel_type]:
                
                word_pairs_grouped_by_rel_type[rel_type].add(f"{word_1}||{word_2}||{similarity}")
                found_rel = True
        # Если для пары слов не нашлось отношения, запоминаем,
        # что отношение между ними не найдено
        if not found_rel:
            if word_pairs_grouped_by_rel_type.get("no_relation") is None:
                word_pairs_grouped_by_rel_type["no_relation"] = set()
            word_pairs_grouped_by_rel_type["no_relation"].add(f"{word_1}||{word_2}||{similarity}")
    for rel_type in word_pairs_grouped_by_rel_type.keys():
        w_pairs_set = word_pairs_grouped_by_rel_type[rel_type]
        w_pairs_split = []
        for pair in w_pairs_set:
            attrs = pair.split('||')
            w_1 = attrs[0]
            w_2 = attrs[1]
            sim = float(attrs[2])
            t = (w_1, w_2, sim)
            w_pairs_split.append(t)
        word_pairs_grouped_by_rel_type[rel_type] = w_pairs_split
            
    return word_pairs_grouped_by_rel_type

In [12]:
similarities_path = "wordsim353_sim_rel/wordsim_similarity_goldstandard.txt"
word_similarity_triples = load_similarity_pairs(inp_path=similarities_path)

In [13]:
vocab = create_vocab_from_similarity_triples(word_similarity_triples)
word_relation_info = find_relation_synsets_for_words(wn, words_list=vocab)

In [14]:
word_pairs_grouped_by_rel_type = group_sim_pairs_by_rel_type(word_similarity_triples,
                                                             word_relation_info)

In [15]:
def group_pairs_info_by_rel_type(word_pairs_grouped_by_rel_type):
    pairs_by_rel_type = {}
    scores_sum_by_rel_type = {}
    pair_count_by_rel_type = {}
    for rel_type, triplets in word_pairs_grouped_by_rel_type.items():
        if len(triplets) == 0:
            continue
        # scores_sum_by_rel_type.get(rel_type) is None:
        #     scores_sum_by_rel_type[rel_type] = []
        #     pair_count_by_rel_type[rel_type] = []
        scores_sum = sum(t[2] for t in triplets)
        pair_count = len(triplets)
        pairs_by_rel_type[rel_type] = [(t[0], t[1]) for t in triplets]
        scores_sum_by_rel_type[rel_type] = scores_sum
        pair_count_by_rel_type[rel_type] = pair_count

    return pairs_by_rel_type, scores_sum_by_rel_type, pair_count_by_rel_type

In [16]:
pairs_by_rel_type, scores_sum_by_rel_type, pair_count_by_rel_type \
        = group_pairs_info_by_rel_type(word_pairs_grouped_by_rel_type)

## Число пар и среднее сходство пар разных типов отношений

In [17]:
for key in scores_sum_by_rel_type.keys():
    scores_sum = scores_sum_by_rel_type[key]
    pair_count = pair_count_by_rel_type[key]
    
    mean_sim = scores_sum / pair_count
    print(f"Тип отношения: {key}, число пар: {pair_count}, средняя близость: {mean_sim:.3f}")

Тип отношения: synonym, число пар: 8, средняя близость: 8.772
Тип отношения: hyponym-hypernym, число пар: 16, средняя близость: 8.156
Тип отношения: other, число пар: 3, средняя близость: 7.563
Тип отношения: no_relation, число пар: 177, средняя близость: 4.673


## Списки пар слов, сгруппированные по типу отношения

In [20]:
for rel_type, pairs in pairs_by_rel_type.items():
    if rel_type != "no_relation":
        print(f"Тип отношения: {rel_type}")
        for (w1, w2) in pairs:
            print(f"\t{w1} - {w2}")

Тип отношения: synonym
	buck - dollar
	gem - jewel
	king - queen
	calculation - computation
	magician - wizard
	automobile - car
	midday - noon
	forest - wood
Тип отношения: hyponym-hypernym
	liquid - water
	journey - voyage
	boy - lad
	cat - tiger
	aluminum - metal
	currency - money
	cat - jaguar
	kind - type
	football - soccer
	example - precedent
	avenue - street
	coast - shore
	psychology - science
	food - seafood
	bird - cock
	asylum - madhouse
Тип отношения: other
	man - woman
	smart - stupid
	king - queen
